In [1]:
import numpy as np
import pickle

In [2]:
def get_active_cells(deconvolved_traces):

    # going to return a dictionary with only active cells, formatted exactly the same as traces

    d = dict.fromkeys(deconvolved_traces.keys())

    for cell in deconvolved_traces:
        if deconvolved_traces[cell]['active'] == True:
            d[cell] = deconvolved_traces[cell]
        else:
            d.pop(cell,None)

    return d

In [3]:
BASE_PATH_1 = "F:/Two-Photon/Psilocybin Project/Evoked Cohort Mice/ID474/psilo/TSeries-10132023-1324-142/suite2p/plane0/"
BASE_PATH_2 = "F:/Two-Photon/Psilocybin Project/Evoked Cohort Mice/ID474/psilo/TSeries-10132023-1324-143/suite2p/plane0/"
file_path = "F:/Two-Photon/Psilocybin Project/Evoked Cohort Mice/ID474/psilo/TSeries-10132023-1324-143/suite2p/plane0/roimatch_142_143.csv"

iscellpath_1 = BASE_PATH_1 + "iscell.npy"
tracepath_1 = BASE_PATH_1 + "F.npy"
cell_dict_1 = BASE_PATH_1 + "cells.pkl"

iscellpath_2 = BASE_PATH_2 + "iscell.npy"
tracepath_2 = BASE_PATH_2 + "F.npy"
cell_dict_2 = BASE_PATH_2 + "cells.pkl"


iscell_1 = np.load(iscellpath_1)
traces_1 = np.load(tracepath_1)

iscell_2 = np.load(iscellpath_2)
traces_2 = np.load(tracepath_2)


with open(cell_dict_1, 'rb') as f:
    cell_dict_1 = pickle.load(f)

with open(cell_dict_2, 'rb') as f:
    cell_dict_2 = pickle.load(f)



In [4]:
ROI_indices_1 = (iscell_1[:,0] == 1).nonzero()
ROI_indices_1 = ROI_indices_1[0] # extracting the first part of the tuple
cell_IDs_1 = ROI_indices_1 + 1 # add 1 so we don't have zero indexing

# print(cell_IDs_1[436])

ROI_indices_2 = (iscell_2[:,0] == 1).nonzero()
ROI_indices_2 = ROI_indices_2[0] # extracting the first part of the tuple
cell_IDs_2 = ROI_indices_2 + 1 # add 1 so we don't have zero indexing

# print(cell_IDs_2[255])

In [5]:


# Load the CSV file into a NumPy array
matched_cells = np.genfromtxt(file_path, delimiter=',', skip_header=0)
matched_cells = matched_cells.astype(int) - 1  # Subtract 1 and convert to int in one step

# Extract columns 0 and 1
column_0 = matched_cells[:, 0]
column_1 = matched_cells[:, 1]

# Retrieve lists using NumPy indexing
cell_array = np.zeros_like(matched_cells)
cell_array[:,0] = cell_IDs_1[column_0]
cell_array[:,1] = cell_IDs_2[column_1]

print("Detected", len(cell_array[:,0]), "matched cells in first and",len(cell_array[:,1]), "in second recording (should be the same)")


active_1 = get_active_cells(cell_dict_1)

matched_active_1 = [cell for cell in cell_array[:,0] if cell in active_1.keys()]

print(len(matched_active_1), "active cells detected in pre- recording")

active_2 = get_active_cells(cell_dict_2)

matched_active_2 = [cell for cell in cell_array[:,1] if cell in active_2.keys()]

print(len(matched_active_2), "active cells detected in post- recording")


Detected 117 matched cells in first and 117 in second recording (should be the same)
46 active cells detected in pre- recording
50 active cells detected in post- recording


In [6]:
#  Make a two column array containing each cell and it's matched cell. 

coactive_cells = []
responded_in_minimum_one = []
became_inactive = []
became_active = []


for i, (cell_1, cell_2) in enumerate(cell_array):
    if cell_1 in matched_active_1 and cell_2 in matched_active_2:
        coactive_cells.append(cell_array[i, :])
    elif cell_1 in matched_active_1 and cell_2 not in matched_active_2:
        became_inactive.append(cell_array[i,0])
    elif cell_1 not in matched_active_1 and cell_2 in matched_active_2:
        became_active.append(cell_array[i,0])
    if cell_1 in matched_active_1 or cell_2 in matched_active_2:
        responded_in_minimum_one.append(cell_array[i, :])

# Convert the list of coactive_cells to a NumPy array
coactive_cells_array = np.array(coactive_cells)
active_only_first_array = np.array(became_inactive)
active_only_second_array = np.array(became_active)
active_in_minimum_one_array = np.array(responded_in_minimum_one)
print("There were", len(coactive_cells_array),"cells responsive in both recordings")
print("There were", len(became_inactive),"cells responsive only in the first recording")
print("There were", len(became_active),"cells responsive only in the second recording")
print("There were", len(responded_in_minimum_one),"cells responsive in at least one recording")

np.save(BASE_PATH_2 + "coactive_cells_array",coactive_cells_array)
np.save(BASE_PATH_2 + "active_only_first",active_only_first_array)
np.save(BASE_PATH_2 + "active_only_second",active_only_first_array)
np.save(BASE_PATH_2 + "matched_cells",cell_array)



There were 31 cells responsive in both recordings
There were 15 cells responsive only in the first recording
There were 19 cells responsive only in the second recording
There were 65 cells responsive in at least one recording


In [7]:
print(coactive_cells_array)

[[  13   40]
 [  16   31]
 [  21   12]
 [  57  565]
 [ 106 1040]
 [ 111  590]
 [ 124  322]
 [ 134  289]
 [ 147  124]
 [ 152  223]
 [ 154   70]
 [ 155  222]
 [ 172  120]
 [ 177  520]
 [ 182  102]
 [ 183  154]
 [ 185  591]
 [ 206   68]
 [ 223  155]
 [ 284   73]
 [ 323  548]
 [ 382  110]
 [ 385  181]
 [ 390  159]
 [ 431   72]
 [ 460  188]
 [ 560  131]
 [ 639  197]
 [ 732  434]
 [ 801  285]
 [1018  278]]


In [8]:
print(matched_active_1)
print(matched_active_2)

[11, 13, 16, 21, 24, 30, 41, 49, 57, 70, 83, 106, 107, 111, 120, 124, 134, 147, 152, 154, 155, 172, 177, 178, 182, 183, 185, 206, 223, 231, 284, 323, 382, 385, 390, 431, 460, 483, 560, 615, 639, 732, 768, 801, 852, 1018]
[1, 264, 40, 31, 12, 490, 116, 565, 79, 370, 164, 456, 1040, 590, 322, 289, 124, 223, 70, 222, 120, 520, 562, 102, 154, 591, 68, 155, 140, 578, 158, 143, 73, 109, 548, 451, 110, 181, 159, 72, 188, 543, 131, 221, 197, 627, 434, 285, 1045, 278]


In [9]:
# Dictionary Additions:

#  Add a section to the post dictionary entitled 'matches' containing the array of matched cells.  
# Don't need more than this as you can work everything out from these. 

for cell_1, cell_2 in zip(cell_dict_1.keys(), cell_dict_2.keys()):
    cell_dict_1[cell_1]['matched_cells'] = cell_array
    cell_dict_2[cell_2]['matched_cells'] = cell_array

with open(BASE_PATH_1+'cells.pkl','wb') as f:
        pickle.dump(cell_dict_1,f)

with open(BASE_PATH_2+'cells.pkl','wb') as f:
        pickle.dump(cell_dict_2,f)
